In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import random
import seaborn as sns
from sklearn import preprocessing
from sklearn import cross_decomposition
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

sns.set()

# Make charts a bit bolder
#sns.set_context("talk")

%matplotlib inline

# Default figure size
sns.set(rc={"figure.figsize": (12, 6)})

# This actually makes autocomplete WAY faster ...
%config Completer.use_jedi = False

# Show only 2 decimals for floating point numbers
pd.options.display.float_format = "{:.2f}".format

sns.set_style('whitegrid')



In [6]:
eth = pd.read_csv('merge.csv')
eth.head()

btc = pd.read_csv('Binance_BTCUSDT_d.csv')

btc.head()
del btc['unix']
del btc['tradecount']

In [7]:
btc.head()

,date,symbol,open,high,low,close,Volume BTC,Volume USDT
0,2021-11-04 00:00:00,BTC/USDT,62896.49,63040.06,62818.60,62992.16,226.71,14260865.02
1,2021-11-03 00:00:00,BTC/USDT,63220.57,63500.00,60382.76,62896.48,43336.09,2713389500.47
2,2021-11-02 00:00:00,BTC/USDT,60911.12,64270.00,60624.68,63219.99,46368.28,2909221038.40
3,2021-11-01 00:00:00,BTC/USDT,61299.81,62437.74,59405.00,60911.11,44687.67,2737282804.74
4,2021-10-31 00:00:00,BTC/USDT,61859.19,62405.30,59945.36,61299.80,39267.64,2396556477.34


In [8]:
eth.head()

,Unnamed: 0,date,open,high,low,close,volume_ETH,volume_USDT,return,cpi,cpi_change
0,0,2017-08-17,303.61,312.18,298.21,302.00,6155.32,1890017.53,NaN,NaN,NaN
1,1,2017-08-18,302.00,311.79,283.94,293.96,9392.79,2815460.24,-0.03,NaN,NaN
2,2,2017-08-19,293.96,299.90,278.00,290.91,2140.08,618248.58,-0.01,NaN,NaN
3,3,2017-08-20,290.91,300.53,282.85,299.10,2469.49,730800.66,0.03,NaN,NaN
4,4,2017-08-21,299.10,346.52,294.60,323.29,5134.54,1661944.36,0.08,NaN,NaN


In [9]:
btc = btc[::-1]
btc.head()



,date,symbol,open,high,low,close,Volume BTC,Volume USDT
1541,2017-08-17,BTC/USDT,4469.93,4485.39,4200.74,4285.08,647.86,2812379.11
1540,2017-08-18,BTC/USDT,4285.08,4371.52,3938.77,4108.37,1178.07,4994493.56
1539,2017-08-19,BTC/USDT,4108.37,4184.69,3850.00,4139.98,371.15,1508239.39
1538,2017-08-20,BTC/USDT,4139.98,4211.08,4032.62,4086.29,463.54,1915636.35
1537,2017-08-21,BTC/USDT,4086.29,4119.62,3911.79,4016.00,685.12,2770592.06


In [13]:
btc['return'] = btc.close.pct_change()
btc['log_return'] = np.log(1 + btc['return'])

In [14]:
btc.head()

,date,symbol,open,high,low,close,Volume BTC,Volume USDT,return,log_return
1541,2017-08-17,BTC/USDT,4469.93,4485.39,4200.74,4285.08,647.86,2812379.11,NaN,NaN
1540,2017-08-18,BTC/USDT,4285.08,4371.52,3938.77,4108.37,1178.07,4994493.56,-0.04,-0.04
1539,2017-08-19,BTC/USDT,4108.37,4184.69,3850.00,4139.98,371.15,1508239.39,0.01,0.01
1538,2017-08-20,BTC/USDT,4139.98,4211.08,4032.62,4086.29,463.54,1915636.35,-0.01,-0.01
1537,2017-08-21,BTC/USDT,4086.29,4119.62,3911.79,4016.00,685.12,2770592.06,-0.02,-0.02


In [16]:
eth['log_return'] = np.log(1+eth['return'])
eth.head()

,Unnamed: 0,date,open,high,low,close,volume_ETH,volume_USDT,return,cpi,cpi_change,log_return
0,0,2017-08-17,303.61,312.18,298.21,302.00,6155.32,1890017.53,NaN,NaN,NaN,NaN
1,1,2017-08-18,302.00,311.79,283.94,293.96,9392.79,2815460.24,-0.03,NaN,NaN,-0.03
2,2,2017-08-19,293.96,299.90,278.00,290.91,2140.08,618248.58,-0.01,NaN,NaN,-0.01
3,3,2017-08-20,290.91,300.53,282.85,299.10,2469.49,730800.66,0.03,NaN,NaN,0.03
4,4,2017-08-21,299.10,346.52,294.60,323.29,5134.54,1661944.36,0.08,NaN,NaN,0.08


In [18]:
del eth['Unnamed: 0']


In [25]:
eth = eth.rename(columns = {'close':'close_ETH', 'open':'open_ETH', 'high':'high_ETH', 
                            'volume_USDT' : 'Volume_USDT_ETH','return':'return_ETH', 'log_return':'log_return_ETH' })

eth.head()

,date,open_ETH,high_ETH,low,close_ETH,volume_ETH,Volume_USDT_ETH,return_ETH,cpi,cpi_change,log_return_ETH
0,2017-08-17,303.61,312.18,298.21,302.00,6155.32,1890017.53,NaN,NaN,NaN,NaN
1,2017-08-18,302.00,311.79,283.94,293.96,9392.79,2815460.24,-0.03,NaN,NaN,-0.03
2,2017-08-19,293.96,299.90,278.00,290.91,2140.08,618248.58,-0.01,NaN,NaN,-0.01
3,2017-08-20,290.91,300.53,282.85,299.10,2469.49,730800.66,0.03,NaN,NaN,0.03
4,2017-08-21,299.10,346.52,294.60,323.29,5134.54,1661944.36,0.08,NaN,NaN,0.08


In [26]:
btc = btc.rename(columns = {'open':'open_BTC', 'high':'high_BTC', 'low':'low_BTC', 'close':'close_BTC', 
                            'Volume USDT': 'Volume_USDT_BTC', 'return':'return_BTC','log_return':'log_return_BTC'
                           })

btc.head()

,date,symbol,open_BTC,high_BTC,low_BTC,close_BTC,Volume BTC,Volume_USDT_BTC,return_BTC,log_return_BTC
1541,2017-08-17,BTC/USDT,4469.93,4485.39,4200.74,4285.08,647.86,2812379.11,NaN,NaN
1540,2017-08-18,BTC/USDT,4285.08,4371.52,3938.77,4108.37,1178.07,4994493.56,-0.04,-0.04
1539,2017-08-19,BTC/USDT,4108.37,4184.69,3850.00,4139.98,371.15,1508239.39,0.01,0.01
1538,2017-08-20,BTC/USDT,4139.98,4211.08,4032.62,4086.29,463.54,1915636.35,-0.01,-0.01
1537,2017-08-21,BTC/USDT,4086.29,4119.62,3911.79,4016.00,685.12,2770592.06,-0.02,-0.02


In [27]:
features = pd.merge(eth, btc, how = 'outer')

features.head()

,date,open_ETH,high_ETH,low,close_ETH,volume_ETH,Volume_USDT_ETH,return_ETH,cpi,cpi_change,log_return_ETH,symbol,open_BTC,high_BTC,low_BTC,close_BTC,Volume BTC,Volume_USDT_BTC,return_BTC,log_return_BTC
0,2017-08-17,303.61,312.18,298.21,302.00,6155.32,1890017.53,NaN,NaN,NaN,NaN,BTC/USDT,4469.93,4485.39,4200.74,4285.08,647.86,2812379.11,NaN,NaN
1,2017-08-18,302.00,311.79,283.94,293.96,9392.79,2815460.24,-0.03,NaN,NaN,-0.03,BTC/USDT,4285.08,4371.52,3938.77,4108.37,1178.07,4994493.56,-0.04,-0.04
2,2017-08-19,293.96,299.90,278.00,290.91,2140.08,618248.58,-0.01,NaN,NaN,-0.01,BTC/USDT,4108.37,4184.69,3850.00,4139.98,371.15,1508239.39,0.01,0.01
3,2017-08-20,290.91,300.53,282.85,299.10,2469.49,730800.66,0.03,NaN,NaN,0.03,BTC/USDT,4139.98,4211.08,4032.62,4086.29,463.54,1915636.35,-0.01,-0.01
4,2017-08-21,299.10,346.52,294.60,323.29,5134.54,1661944.36,0.08,NaN,NaN,0.08,BTC/USDT,4086.29,4119.62,3911.79,4016.00,685.12,2770592.06,-0.02,-0.02


In [28]:
features.to_csv('features.csv')